# [$\phi$] `STARSKØPE`

<pre><i class="fa fa-android"></i><i class="fa fa-flask "></i><i class="fa fa-rocket"></i> A Deep Neural Network Cyberoptic Telescope for Astrophysical Classification of Stars and Planets.</pre>

# Import

In [3]:
# Import libraries
import pandas as pd
import numpy as np

# default style/formatting
import matplotlib as mpl
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
plt.style.use('seaborn-bright')

font_dict={'family':'TitilliumWeb',
          'size':16}
mpl.rc('font',**font_dict)

In [21]:
# astro.py for extremely useful astronomical tools
# uncomment items below to install if needed:

# !pip install astropy astroquery
# !pip install astropy spectral-cube
# !pip install astropy reproject


# Many of the notebooks make use of ImageFileCollections in ccdproc to inspect the image header. To install this, type:

#!pip install astropy ccdproc


import astropy.units as u

## Libraries

Required Imports
- `astroquery.mast Observations`: Establishes a connection to a server to query MAST. Please try re-running the cell if the connection fails.
- `os`: Python interface to the operating system.
- `shutil`: Python shell utilities.
- `stwcs`: HST world coordinate system (WCS) updates.

In [15]:
# set path for retrieving and storing files

import os
import shutil
import sys

In [18]:
%pwd

'/Users/hakkeray/CODE/CAPSTONE/starskope/notebooks'

In [19]:
%ls

 kepler3-exodata.ipynb               fast-fourier-transform.ipynb
FITS-image-import-astropy.ipynb      hex_coords.html
Icon?                                hubble-hst-aws-api.ipynb
MAST-Astroquery.ipynb                k2_tensorflow.ipynb
Normalise/                           starskøpe-0.ipynb
UVES-FITs.ipynb                      starskøpe_5_uvesSpectroscopy.ipynb
astropy_Time.ipynb                   starskøpe_6_FITS-cubes.ipynb


# Datasets

## K2 Timeseries Photometry

In [ ]:
# https://stsci.app.box.com/s/4no7430kswla4gsg8bt2avs72k9agpne

# MAST has notebooks with astroquery/API examples:
# •https://github.com/spacetelescope/notebooks/tree/master/notebooks/MASTAstroqueryDocumentation
# •https://astroquery.readthedocs.io/en/latest/mast/mast.html
# •https://astroquery.readthedocs.io/en/latest/Datasets 
# on the AWS Cloud 
# •https://mast-labs.stsci.io/2018/06/hst-public-data-on-aws

# MAST Slack Channel
# MAST Helpdesk: archive@stsci.edu

In [22]:

# train_df = pd.read_csv('exoTrain.csv')
# test_df = pd.read_csv('exoTest.csv')

# Scrub

In [ ]:
train_df.shape
test_df.shape

In [ ]:
# checking for imbalanced classes
train_df['LABEL'].value_counts()
test_df['LABEL'].value_counts()

In [17]:
# look at the first Star in the dataset
star0 = train_df.iloc[0, :]
star0.head()

In [7]:
# Scatter Plot For First Star
plt.figure(figsize=(15, 5))
plt.scatter(pd.Series([i for i in range(1, len(star0))]), star0[1:])
plt.ylabel('Flux')
plt.xlabel('Time')
plt.title('Flux for Star 0 - scatterplot')
plt.show()

In [ ]:
# Line Plot For First Star
plt.figure(figsize=(15, 5))
plt.plot(pd.Series([i for i in range(1, len(star0))]), star0[1:])
plt.ylabel('Flux')
plt.xlabel('Time')
plt.title('Flux for Star 0 - line plot')
plt.show()

In [5]:


from astropy.utils.data import download_file
from astropy.io import fits  # We use fits to open the actual data file
from astropy.utils import data
data.conf.remote_timeout = 60
from spectral_cube import SpectralCube
from astroquery.esasky import ESASky
from astroquery.utils import TableList
from astropy.wcs import WCS
from reproject import reproject_interp


 Time Series (at varying $\phi$)📓

In [ ]:
# Converting times
# `astropy.time <http://docs.astropy.org/en/stable/time/index.html>`__ provides methods to convert times and dates between different systems and formats. Since the ESO FITS headers already contain the time of the observation in different systems, we could just read the keyword in the time system we like, but we will use astropy.time to make this conversion here. astropy.time.Time will parse many common input formats (strings, floats), but unless the format is unambiguous the format needs to be specified (e.g. a number could mean JD or MJD or year). Also, the time system needs to be given (e.g. UTC). Below are several examples, initialized from different header keywords.



from astropy.time import Time
t1 = Time(header['MJD-Obs'], format = 'mjd', scale = 'utc')
t2 = Time(header['Date-Obs'], scale = 'utc')

#Times can be expressed in different formats:

t1
t1.isot
t2

#can be converted to a different time system.
t1.tt

#<Time object: scale='tt' format='mjd' value=55784.97567650852>

#Times can also be initialized from arrays and we can calculate time differences.

obs_times = Time(date, scale = 'utc')
delta_t = obs_times - Time(date[0], scale = 'utc')


#Now we want to express the time difference between the individual spectra of MN Lup in rotational periods. While the unit of delta_t is days, unfortunately astropy.time.Time and astropy.units.Quantity objects don’t work together yet, so we’ll have to convert from one to the other explicitly.


delta_p = delta_t.value * u.day / period

In [ ]:
# Normalize the flux to the local continuum
# In this example we want to look at the time evolution of a single specific emission line in the spectrum. In order to estimate the equivalent width or make reasonable plots we need to normalize the flux to the local continuum. In this specific case the emission line is bright and the continuum can be described reasonably by a second-order polynomial.

# So, we define two regions left and right of the emission line, where we fit the polynomial. Looking at the figure, [3925*u.AA, 3930*u.AA] and [3938*u.AA, 3945*u.AA] seem right for that. Then, we normalize the flux by this polynomial.

# The following function will do that:



def region_around_line(w, flux, cont):
    '''cut out and normalize flux around a line

    Parameters
    ----------
    w : 1 dim np.ndarray array of wavelengths
    flux : np.ndarray of shape (N, len(w))
    array of flux values for different spectra in the series
    cont : list of lists
    wavelengths for continuum normalization [[low1,up1],[low2, up2]]
    that described two areas on both sides of the line
    '''
    #index is true in the region where we fit the polynomial
    indcont = ((w > cont[0][0]) & (w < cont[0][1])) |((w > cont[1][0]) & (w < cont[1][1]))
    #index of the region we want to return
    indrange = (w > cont[0][0]) & (w < cont[1][1])
    # make a flux array of shape
    # (number of spectra, number of points in indrange)
    f = np.zeros((flux.shape[0], indrange.sum()))
    for i in range(flux.shape[0]):
        # fit polynomial of second order to the continuum region
        linecoeff = np.polyfit(w[indcont], flux[i, indcont], 2)
        # divide the flux by the polynomial and put the result in our
        # new flux array
        f[i,:] = flux[i,indrange] / np.polyval(linecoeff, w[indrange].value)
    return w[indrange], f

wcaII, fcaII = region_around_line(wavelength, flux,
    [[3925*u.AA, 3930*u.AA],[3938*u.AA, 3945*u.AA]])

# Query AWS to S3 Storage

In [ ]:
# This script queries MAST for WFC3 IR data and downloads the data from 
# the AWS public dataset rather than from MAST servers.

# Working with http://astroquery.readthedocs.io/en/latest/mast/mast.html
from astroquery.mast import Observations
import boto3

# This downloads all the F160W DRZ images from CANDELS in the GOODS-South field
obsTable = Observations.query_criteria(project='HST',
                                       filters='F160W',
                                       instrument_name='WFC3/IR',
                                       proposal_id=['12062','12061','12062'])

# Grab the first 100 records
products = Observations.get_product_list(obsTable[:100])

# Select only drizzled (DRZ) files
filtered = Observations.filter_products(products,
                                        mrp_only=False,
                                        productSubGroupDescription='DRZ')

# Enable 'S3 mode' for module which will return S3-like URLs for FITs files
# e.g. s3://stpubdata/hst/public/icde/icde43l0q/icde43l0q_drz.fits
Observations.enable_s3_hst_dataset()

# Grab the S3 URLs for each of the observations
s3_urls = Observations.get_hst_s3_uris(filtered)

s3 = boto3.resource('s3')

# Create an authenticated S3 session. Note, download within US-East is free
# e.g. to a node on EC2.
s3_client = boto3.client('s3',
                         aws_access_key_id='YOURAWSACCESSKEY',
                         aws_secret_access_key='YOURSECRETACCESSKEY')

bucket = s3.Bucket('stpubdata')

# Just download a few of the files (remove the [0:3] to download them all)
for url in s3_urls[0:3]:
  # Extract the S3 key from the S3 URL
  fits_s3_key = url.replace("s3://stpubdata/", "")
  root = url.split('/')[-1]
  bucket.download_file(fits_s3_key, root, ExtraArgs={"RequestPayer": "requester"})